In [1]:
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent.parent.absolute()))
import config

In [2]:
from context import Context
from utils.utils_chain import WrapperAddress as Address, Account, hex_to_string
from utils.utils_chain import nominated_amount

context = Context()

2024-01-08 14:46:30,169 - WARNING - Structure definition for stakings_boosted not found in /home/ovi/projects/dex/mx-sdk-py-exchange/deploy/configs-testnet/deploy_structure.json
2024-01-08 14:46:30,170 - WARNING - Structure definition for metastakings_boosted not found in /home/ovi/projects/dex/mx-sdk-py-exchange/deploy/configs-testnet/deploy_structure.json
2024-01-08 14:46:30,170 - WARNING - Structure definition for position_creator not found in /home/ovi/projects/dex/mx-sdk-py-exchange/deploy/configs-testnet/deploy_structure.json


loaded 10 accounts from 1 PEM files.


2024-01-08 14:46:30,691 - DEBUG - Account.sync_nonce() done: 324
2024-01-08 14:46:31,611 - DEBUG - Account.sync_nonce() done: 3
2024-01-08 14:46:31,620 - INFO - Loaded 5 tokens.
2024-01-08 14:46:32,598 - DEBUG - Account.sync_nonce() done: 324
2024-01-08 14:46:32,600 - INFO - Loaded 3 egld_wraps.
2024-01-08 14:46:32,602 - INFO - Loaded 1 locked_assets.
2024-01-08 14:46:32,603 - INFO - Loaded 1 proxies.
2024-01-08 14:46:32,606 - INFO - Loaded 1 simple_locks.
2024-01-08 14:46:32,607 - INFO - Loaded 1 simple_locks_energy.
2024-01-08 14:46:32,608 - INFO - Loaded 1 fees_collectors.
2024-01-08 14:46:32,609 - INFO - Loaded 1 unstakers.
2024-01-08 14:46:32,610 - INFO - Loaded 1 proxies_v2.
2024-01-08 14:46:32,612 - INFO - Loaded 0 router.
2024-01-08 14:46:32,614 - INFO - Loaded 1 router_v2.
2024-01-08 14:46:32,616 - INFO - Loaded 0 pairs.
2024-01-08 14:46:32,618 - INFO - Loaded 4 pairs_v2.
2024-01-08 14:46:32,620 - INFO - Loaded 1 pairs_view.
2024-01-08 14:46:32,621 - INFO - Loaded 0 farms_comm

In [4]:
from contracts.farm_contract import FarmContract
from utils.contract_retrievers import retrieve_farm_by_address
# farm_contracts = context.get_contracts(config.FARMS_V2)

In [4]:
farm_contract: FarmContract = context.deploy_structure.get_deployed_contract_by_index(config.FARMS_V2, 2)

In [ ]:
farm_contract: FarmContract = retrieve_farm_by_address("")

set new rewards per block

In [ ]:
farm_contract.set_rewards_per_block(context.deployer_account, context.network_provider.proxy, nominated_amount(14000))

start contracts

In [ ]:
farm_contract.resume(context.deployer_account, context.network_provider.proxy)

produce rewards

In [ ]:
farm_contract.start_produce_rewards(context.deployer_account, context.network_provider.proxy)

upgrade contract

In [5]:
pair_address = farm_contract.get_lp_address(context.network_provider.proxy)
tx_hash = farm_contract.contract_upgrade(context.deployer_account, context.network_provider.proxy, 
                                         config.FARM_V2_BYTECODE_PATH, [pair_address, context.deployer_account.address.bech32()])

2023-12-05 16:42:21,742 - INFO - upgrade FarmContract contract
2023-12-05 16:42:21,744 - DEBUG - Arguments: ['MEX-455c57', 'EGLDRIDE-7bd51a', 1000000000000, erd1qqqqqqqqqqqqqpgqav09xenkuqsdyeyy5evqyhuusvu4gl3t2jpss57g8x, erd1ss6u80ruas2phpmr82r42xnkd6rxy40g9jl69frppl4qez9w2jpsqj8x97, erd1ss6u80ruas2phpmr82r42xnkd6rxy40g9jl69frppl4qez9w2jpsqj8x97]
2023-12-05 16:42:21,744 - DEBUG - Upgrade FarmContract contract
2023-12-05 16:42:22,610 - DEBUG - Upgrade arguments: ['MEX-455c57', 'EGLDRIDE-7bd51a', 1000000000000, erd1qqqqqqqqqqqqqpgqav09xenkuqsdyeyy5evqyhuusvu4gl3t2jpss57g8x, erd1ss6u80ruas2phpmr82r42xnkd6rxy40g9jl69frppl4qez9w2jpsqj8x97, erd1ss6u80ruas2phpmr82r42xnkd6rxy40g9jl69frppl4qez9w2jpsqj8x97]
2023-12-05 16:42:23,253 - INFO - No explorer known for https://proxy-shadowfork-four.elrond.ro. transaction raw path: transactions/0d2d5f1a6aef2cdc18fc785ef400ddcc9296f18eed03d4bfcdf11f08daeb8d8a


In [5]:
from contracts.farm_contract import FarmContractVersion
context.deployer_account.sync_nonce(context.network_provider.proxy)
# upgrade all farms
farms : list[FarmContract] = context.get_contracts(config.FARMS_V2)
for farm in farms:
    farm.version = FarmContractVersion.V2Boosted
    pair = farm.get_lp_address(context.network_provider.proxy)
    tx_hash = farm.contract_upgrade(context.deployer_account, context.network_provider.proxy, 
                                         config.FARM_V3_BYTECODE_PATH, [], no_init=True)
    context.network_provider.check_complex_tx_status(tx_hash, "farm upgrade")

2024-01-08 15:11:26,326 - DEBUG - Account.sync_nonce() done: 324
2024-01-08 15:11:27,613 - INFO - upgrade FarmContract contract
2024-01-08 15:11:27,614 - DEBUG - Arguments: []
2024-01-08 15:11:27,615 - DEBUG - Upgrade FarmContract contract
2024-01-08 15:11:28,075 - DEBUG - Upgrade arguments: []
2024-01-08 15:11:29,221 - INFO - View this transaction in the MultiversX Testnet Explorer: https://testnet-explorer.multiversx.com/transactions/dedf3c561ccff5c71e644e1bfcd609ddc785f404c60d7015d5a18a8b686351cd
2024-01-08 15:11:29,222 - DEBUG - Waiting for transaction dedf3c561ccff5c71e644e1bfcd609ddc785f404c60d7015d5a18a8b686351cd to be executed...
2024-01-08 15:11:35,520 - DEBUG - Transaction dedf3c561ccff5c71e644e1bfcd609ddc785f404c60d7015d5a18a8b686351cd status: success
2024-01-08 15:11:35,521 - DEBUG - TX status most likely false success, making sure...
2024-01-08 15:11:42,149 - DEBUG - Making sure tx status is not false success...
2024-01-08 15:11:48,402 - DEBUG - Transaction dedf3c561ccff5c

In [6]:
# resume all farms
context.deployer_account.sync_nonce(context.network_provider.proxy)
contracts : list[FarmContract] = context.get_contracts(config.FARMS_V2)
for contract in contracts:
    tx_hash = contract.resume(context.deployer_account, context.network_provider.proxy)

2024-01-08 15:13:02,284 - DEBUG - Account.sync_nonce() done: 328
2024-01-08 15:13:02,285 - INFO - Resume farm contract
2024-01-08 15:13:02,286 - DEBUG - Calling resume at erd1qqqqqqqqqqqqqpgqk232ukl7944vmjq44krjns0yturh6ms9exksgmr2sc
2024-01-08 15:13:02,287 - DEBUG - Args: []
2024-01-08 15:13:02,607 - DEBUG - Contract call arguments: []
2024-01-08 15:13:03,382 - INFO - View this transaction in the MultiversX Testnet Explorer: https://testnet-explorer.multiversx.com/transactions/ab7f19e77e0fa73e37bc2de8ccdee51a2a01d291e39e588402b106a8d16850af
2024-01-08 15:13:03,383 - INFO - Resume farm contract
2024-01-08 15:13:03,384 - DEBUG - Calling resume at erd1qqqqqqqqqqqqqpgq3qw8j5kd6295y8ud4kwc7k3qyhdcvu6fexksas5lm6
2024-01-08 15:13:03,385 - DEBUG - Args: []
2024-01-08 15:13:03,669 - DEBUG - Contract call arguments: []
2024-01-08 15:13:03,829 - INFO - View this transaction in the MultiversX Testnet Explorer: https://testnet-explorer.multiversx.com/transactions/42231a356431ceadc3a37df37956bdd484

change owner

In [ ]:
tx_hash = farm_contract.update_owner_or_admin(context.deployer_account, context.network_provider.proxy, "erd19uysnnzwnzm3klq8qzv7zprw2d3yj38hx0ac9cuqvl0j2jr5kp2sflvmyv")

resume contract

In [6]:
farm_contract.resume(context.deployer_account, context.network_provider.proxy)

2023-12-05 18:19:06,601 - INFO - Resume farm contract
2023-12-05 18:19:06,604 - DEBUG - Calling resume at erd1qqqqqqqqqqqqqpgqenvn0s3ldc94q2mlkaqx4arj3zfnvnmakp2sxca2h9
2023-12-05 18:19:06,605 - DEBUG - Args: []
2023-12-05 18:19:07,668 - DEBUG - Contract call arguments: []
2023-12-05 18:19:08,587 - INFO - No explorer known for https://proxy-shadowfork-four.elrond.ro. transaction raw path: transactions/1d2612897558bccd14e6f93012b03e86826dc692f555e4a7e0dbfbe64f4dea83


'1d2612897558bccd14e6f93012b03e86826dc692f555e4a7e0dbfbe64f4dea83'

Collect user accounts

In [7]:
from utils.utils_scenarios import collect_farm_contract_users
from multiversx_sdk_network_providers import ApiNetworkProvider

mainnet_api = ApiNetworkProvider("https://api.multiversx.com")
fetched_users = collect_farm_contract_users(100, farm_contract.address, farm_contract.farmingToken, farm_contract.farmToken,
                                            mainnet_api, context.network_provider.proxy)

2024-01-04 16:59:53,993 - INFO - Collecting users from farm contract erd1qqqqqqqqqqqqqpgqenvn0s3ldc94q2mlkaqx4arj3zfnvnmakp2sxca2h9 ...
2024-01-04 16:59:55,275 - DEBUG - Processing user erd177ns4nd8n2xj76nzq69kk3rdrm0nm6ayw5zd8x77p4c3ulyje4cqznkywy ...
2024-01-04 16:59:55,276 - DEBUG - Processing user erd1vup5z3jqag039xl8xasv7m7wplnu0kvnzdfxywf4jls50s0stzls3htgsk ...
2024-01-04 16:59:55,281 - DEBUG - Processing user erd1r5hsrz5z3kv9fyk64xn685g4u9kq2a5tjqe76tagrmast09r8fxqsdvmul ...
2024-01-04 16:59:55,282 - DEBUG - Processing user erd1jp0s6rngshhzvyrhxr5hra9pwgyspa2g2unfwjwppxdus36ydgfqtftae4 ...
2024-01-04 16:59:55,283 - DEBUG - Processing user erd1xyff74cpvv5jumsu6sps8fksjay03ls4lcz05mhx3awq4uuje4tqrqy6zk ...
2024-01-04 16:59:55,285 - DEBUG - Processing user erd1jc0wf7hn7pg8yxwealrhn0wzpcfhys6azvuemgxkvxmkjf07sqhsppehs4 ...
2024-01-04 16:59:55,286 - DEBUG - Processing user erd1v5rrgarnx72wjp8cfgwlfa77460a6td0qhcvhu4jkv25tnls5amseek0j2 ...
2024-01-04 16:59:55,288 - DEBUG - Processing 

Used users

In [7]:
from utils.decoding_structures import FARM_TOKEN_ATTRIBUTES
from utils.utils_chain import decode_merged_attributes, base64_to_hex, get_all_token_nonces_details_for_account
from utils.utils_scenarios import FetchedUser

index = 0
users: List[FetchedUser] = fetched_users.get_users_with_farm_tokens()
if not users:
    raise Exception('No users found for the given criteria')

user = users[index].address
tokens_in_account = get_all_token_nonces_details_for_account(farm_contract.farmingToken, user.bech32(), context.network_provider.proxy)
print(f'Account: {user.bech32()}')
print(f'Farming Tokens in account:')
for token in tokens_in_account:
    print(f'\t{token}')
tokens_in_account = get_all_token_nonces_details_for_account(farm_contract.farmToken, user.bech32(), context.network_provider.proxy)
print(f'Farm Tokens in account:')
for token in tokens_in_account:
    print(f'\t{token}')
    print(f'\t\t{decode_merged_attributes(base64_to_hex(token["attributes"]), FARM_TOKEN_ATTRIBUTES)}')

Account: erd124tqyjf5035vcyjzr467qkezcmkus84f8hhxrj20wxy23m8xm88qklcdfc
Farming Tokens in account:
Farm Tokens in account:
	{'attributes': 'AAAACAgN9EUft74AAAAAAAAAA4AAAAAAAAAACAHK6zqFwTNNVVYCSTR8aMwSQh114Fsixu3IHqk97mHJT3GIqOzm2c4=', 'balance': '129174175918142285', 'creator': 'erd1qqqqqqqqqqqqqpgqenvn0s3ldc94q2mlkaqx4arj3zfnvnmakp2sxca2h9', 'nonce': 54951, 'royalties': '0', 'tokenIdentifier': 'EGLDRIDEFL-7aac94-d6a7', 'uris': ['']}
		{'reward_per_share': 580388504722718208, 'entering_epoch': 896, 'compounded_reward': 0, 'current_farm_amount': 129174175918142285, 'original_owner': 'erd124tqyjf5035vcyjzr467qkezcmkus84f8hhxrj20wxy23m8xm88qklcdfc'}


In [8]:
from utils.decoding_structures import FARM_TOKEN_ATTRIBUTES
from utils.utils_chain import decode_merged_attributes, base64_to_hex
index = 1
users: List[FetchedUser] = fetched_users.get_users_with_farm_tokens()
if not users:
    raise Exception('No users found for the given criteria')

user2 = users[index].address
tokens_in_account = get_all_token_nonces_details_for_account(farm_contract.farmingToken, user2.bech32(), context.network_provider.proxy)
print(f'Account: {user2.bech32()}')
print(f'Farming Tokens in account:')
for token in tokens_in_account:
    print(f'\t{token}')
tokens_in_account = get_all_token_nonces_details_for_account(farm_contract.farmToken, user2.bech32(), context.network_provider.proxy)
print(f'Farm Tokens in account:')
for token in tokens_in_account:
    print(f'\t{token}')
    print(f'\t\t{decode_merged_attributes(base64_to_hex(token["attributes"]), FARM_TOKEN_ATTRIBUTES)}')

Account: erd1c2tt2z8lflwgp63zy9zmpyqkgg95yq6hl6v9eax45ymn35c40g2qe0unq3
Farming Tokens in account:
Farm Tokens in account:
	{'attributes': 'AAAACAMq3AD5NNVWAAAAAAAAA4sAAAAAAAAACA47ylNDzD3Iwpa1CP9P3IDqIiFFsJAWQgtCA1f+mFz01aE3ONMVehQ=', 'balance': '1025635799125474760', 'creator': 'erd1qqqqqqqqqqqqqpgqenvn0s3ldc94q2mlkaqx4arj3zfnvnmakp2sxca2h9', 'nonce': 24263, 'royalties': '0', 'tokenIdentifier': 'EGLDRIDEFL-7aac94-5ec7', 'uris': ['']}
		{'reward_per_share': 228236627874731350, 'entering_epoch': 907, 'compounded_reward': 0, 'current_farm_amount': 1025635799125474760, 'original_owner': 'erd1c2tt2z8lflwgp63zy9zmpyqkgg95yq6hl6v9eax45ymn35c40g2qe0unq3'}


Transfer position

In [ ]:
from utils.utils_tx import multi_esdt_transfer, ESDTToken
from utils.utils_chain import Account
fetched_user = users[0]
user_account = Account(pem_file="~/Documents/sh1.pem")
user_account.address = fetched_user.address
user_account.sync_nonce(context.network_provider.proxy)

full_token_id = fetched_user.farm_tokens[0].get('tokenIdentifier')
token_id = '-'.join(full_token_id.split('-')[:-1])
token = ESDTToken(token_id, fetched_user.farm_tokens[0].get('nonce'), int(fetched_user.farm_tokens[0].get('balance'))//2)
print(f'Sending: {token.get_token_data()} from {fetched_user.address.bech32()}')
multi_esdt_transfer(context.network_provider.proxy, 1000000, user_account, users[1].address, [token])

Enter farm

In [ ]:
from utils.utils_chain import Account, WrapperAddress as Address, get_all_token_nonces_details_for_account
from contracts.farm_contract import EnterFarmEvent

# user = fetched_users[randint(0, len(fetched_users.get_users_with_farming_tokens()) - 1)].address
# user = fetched_users.get_users_with_both_tokens()[0].address
user_account = Account(pem_file="~/Documents/sh1.pem")
user_account.address = user
user_account.sync_nonce(context.network_provider.proxy)

farming_tk_balance = 0
tokens_in_account = get_all_token_nonces_details_for_account(farm_contract.farmingToken, user_account.address.bech32(), context.network_provider.proxy)
print(f'Found {len(tokens_in_account)} farming tokens in account')
for token in tokens_in_account:
    if int(token['balance']) > 0:
        farming_tk_balance = int(token['balance'])
        break

if not farming_tk_balance:
    raise Exception("Not enough farming token balance")

farm_tk_balance, farm_tk_nonce = 0, 0
tokens_in_account = get_all_token_nonces_details_for_account(farm_contract.farmToken, user_account.address.bech32(), context.network_provider.proxy)
print(f'Found {len(tokens_in_account)} farm tokens in account')
for token in tokens_in_account:
    if int(token['balance']) > farm_tk_balance:
        farm_tk_balance = int(token['balance'])
        farm_tk_nonce = token['nonce']
        break

# use only 10% of the balance
event = EnterFarmEvent(farm_contract.farmingToken, 0, farming_tk_balance // 10,
                       farm_contract.farmToken, farm_tk_nonce, farm_tk_balance)

initial_enter: bool = False if farm_tk_nonce else True
tx_hash = farm_contract.enterFarm(context.network_provider, user_account, event, initial=initial_enter)

Claim rewards

In [19]:
from utils.utils_chain import Account, WrapperAddress as Address, get_all_token_nonces_details_for_account
from contracts.farm_contract import ClaimRewardsFarmEvent

# user = fetched_users[randint(0, len(fetched_users.get_users_with_farming_tokens()) - 1)].address
# user = fetched_users.get_users_with_both_tokens()[0].address
user_account = Account(pem_file="~/Documents/sh1.pem")
user_account.address = user
user_account.sync_nonce(context.network_provider.proxy)

farm_tk_balance, farm_tk_nonce = 0, 0
tokens_in_account = get_all_token_nonces_details_for_account(farm_contract.farmToken, user_account.address.bech32(), context.network_provider.proxy)
print(f'Found {len(tokens_in_account)} positions of {farm_contract.farmToken} in account')
for token in tokens_in_account:
    if int(token['balance']) > farm_tk_balance:
        farm_tk_balance = int(token['balance'])
        farm_tk_nonce = token['nonce']
        break

if not farm_tk_nonce:
    raise Exception("Not enough farm token balance")

event = ClaimRewardsFarmEvent(1000000000, farm_tk_nonce, '')

tx_hash = farm_contract.claimRewards(context.network_provider, user_account, event)

2023-12-07 14:45:16,888 - DEBUG - Account.sync_nonce() done: 148
2023-12-07 14:45:19,924 - INFO - claimRewards
2023-12-07 14:45:19,926 - DEBUG - Account: erd124tqyjf5035vcyjzr467qkezcmkus84f8hhxrj20wxy23m8xm88qklcdfc
2023-12-07 14:45:19,927 - DEBUG - claimRewards


Found 1 positions of EGLDRIDEFL-7aac94 in account


2023-12-07 14:45:22,290 - DEBUG - Contract call arguments: []
2023-12-07 14:45:24,664 - INFO - No explorer known for https://proxy-shadowfork-four.elrond.ro. transaction raw path: transactions/4125f0f2d6cba22c7bdceac5fd80326cf6fa8a99b8e241d9889afffcebd26216


In [18]:
from utils.utils_chain import Account, WrapperAddress as Address, get_all_token_nonces_details_for_account
from contracts.farm_contract import ClaimRewardsFarmEvent

user_account = Account(pem_file="~/Documents/sh1.pem")
user_account.address = user
user_account.sync_nonce(context.network_provider.proxy)

event = ClaimRewardsFarmEvent(1000000000, farm_tk_nonce, '', user_account.address.bech32())

tx_hash = farm_contract.claim_boosted_rewards(context.network_provider, user_account, event)

2023-12-06 15:38:15,120 - DEBUG - Account.sync_nonce() done: 1247
2023-12-06 15:38:15,121 - INFO - claimBoostedRewards
2023-12-06 15:38:15,122 - DEBUG - Account: erd195g7t6xwdfcnu6tvye4ks3r09sluzfxhaa7wrd9edqcfu9r645eq8x53v6 claiming for erd195g7t6xwdfcnu6tvye4ks3r09sluzfxhaa7wrd9edqcfu9r645eq8x53v6
2023-12-06 15:38:15,123 - DEBUG - Calling claimBoostedRewards at erd1qqqqqqqqqqqqqpgqenvn0s3ldc94q2mlkaqx4arj3zfnvnmakp2sxca2h9
2023-12-06 15:38:15,124 - DEBUG - Args: [erd195g7t6xwdfcnu6tvye4ks3r09sluzfxhaa7wrd9edqcfu9r645eq8x53v6]
2023-12-06 15:38:15,702 - DEBUG - Contract call arguments: [erd195g7t6xwdfcnu6tvye4ks3r09sluzfxhaa7wrd9edqcfu9r645eq8x53v6]
2023-12-06 15:38:15,906 - INFO - No explorer known for https://proxy-shadowfork-four.elrond.ro. transaction raw path: transactions/49a5a729a3e742ec73f9bed2e92971b82daaaec4e03b18659ab4576595c6adec


Get user farm position

In [16]:
# user_address = fetched_users.get_users_with_both_tokens()[0].address.bech32()
user_address = user.bech32()
print(user_address)
farm_contract.get_user_total_farm_position(user_address, context.network_provider.proxy)

erd195g7t6xwdfcnu6tvye4ks3r09sluzfxhaa7wrd9edqcfu9r645eq8x53v6


{'total_farm_position': 1000000000, 'allow_external_claim': 0}

Exit rewards

In [ ]:
from utils.utils_chain import Account, WrapperAddress as Address, get_all_token_nonces_details_for_account
from contracts.farm_contract import ExitFarmEvent

# user = fetched_users[randint(0, len(fetched_users.get_users_with_farming_tokens()) - 1)].address
# user = fetched_users.get_users_with_both_tokens()[0].address
user_account = Account(pem_file="~/Documents/sh1.pem")
user_account.address = user
user_account.sync_nonce(context.network_provider.proxy)

farm_tk_balance, farm_tk_nonce = 0, 0
tokens_in_account = get_all_token_nonces_details_for_account(farm_contract.farmToken, user_account.address.bech32(), context.network_provider.proxy)
print(f'Found {len(tokens_in_account)} positions of {farm_contract.farmToken} in account')
for token in tokens_in_account:
    if int(token['balance']) > farm_tk_balance:
        farm_tk_balance = int(token['balance'])
        farm_tk_nonce = token['nonce']
        break

if not farm_tk_nonce:
    raise Exception("Not enough farm token balance")

event = ExitFarmEvent(farm_contract.farmToken, farm_tk_balance, farm_tk_nonce, '')

tx_hash = farm_contract.exitFarm(context.network_provider, user_account, event)

Get user farm position

In [ ]:
# user_address = fetched_users.get_users_with_both_tokens()[0].address.bech32()
user_address = user_account.address.bech32()
print(user_address)
farm_contract.get_user_total_farm_position(user_address, context.network_provider.proxy)

SHADOWFORK CONTROL

In [14]:
from contracts.builtin_contracts import SFControlContract

epocs_to_advance = 7

sf_control_contract = SFControlContract(config.SF_CONTROL_ADDRESS)
current_epoch = context.network_provider.proxy.get_network_status(1).epoch_number
print(f'Current epoch: {current_epoch}')
context.deployer_account.sync_nonce(context.network_provider.proxy)
sf_control_contract.epochs_fast_forward(context.deployer_account, context.network_provider.proxy, epocs_to_advance, 9)

Current epoch: 1223


2023-12-06 15:29:34,353 - DEBUG - Account.sync_nonce() done: 3429
2023-12-06 15:29:34,362 - INFO - fast forward epoch
2023-12-06 15:29:34,363 - INFO - Fast forwarding 7 epochs
2023-12-06 15:29:34,363 - DEBUG - Calling epochsFastForward at erd1qqqqqqqqqqqqqqqpqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqplllst77y4l
2023-12-06 15:29:34,364 - DEBUG - Args: [7, 9]
2023-12-06 15:29:35,815 - DEBUG - Contract call arguments: [7, 9]
2023-12-06 15:29:36,600 - INFO - No explorer known for https://proxy-shadowfork-four.elrond.ro. transaction raw path: transactions/012283b2cdb8f8e4404add51a65b580ac6be3fb373b36e98cc50d6f022b3ac34


'012283b2cdb8f8e4404add51a65b580ac6be3fb373b36e98cc50d6f022b3ac34'

wait for epoch fast forward

In [15]:
from time import sleep

target_epoch = current_epoch + epocs_to_advance
while current_epoch < target_epoch:
    current_epoch = context.network_provider.proxy.get_network_status(1).epoch_number
    print(f'Current epoch: {current_epoch}')
    sleep(10)
print('Reached target epoch!')

Current epoch: 1223
Current epoch: 1223
Current epoch: 1223
Current epoch: 1223
Current epoch: 1223
Current epoch: 1224
Current epoch: 1224
Current epoch: 1224
Current epoch: 1224
Current epoch: 1224
Current epoch: 1225
Current epoch: 1225
Current epoch: 1225
Current epoch: 1225
Current epoch: 1225
Current epoch: 1226
Current epoch: 1226
Current epoch: 1226
Current epoch: 1226
Current epoch: 1226
Current epoch: 1227
Current epoch: 1227
Current epoch: 1227
Current epoch: 1227
Current epoch: 1227
Current epoch: 1228
Current epoch: 1228
Current epoch: 1228
Current epoch: 1228
Current epoch: 1228
Current epoch: 1229
Current epoch: 1229
Current epoch: 1229
Current epoch: 1229
Current epoch: 1229
Current epoch: 1230
Reached target epoch!


Trackers

In [ ]:
from trackers.farm_economics_tracking import FarmEconomics
from contracts.farm_contract import FarmContractVersion

farm_tracker = FarmEconomics(farm_contract.address, FarmContractVersion.V2Boosted, context.network_provider)

In [ ]:
farm_tracker.update_tracking_data()
farm_tracker.report_current_tracking_data()

In [ ]:
farm_tracker.update_tracking_data()
farm_tracker.report_current_tracking_data()

State

In [11]:
current_week = farm_contract.get_current_week(context.network_provider.proxy)
print(current_week)

53


In [25]:
energy_for_user_in_week = farm_contract.get_user_energy_for_week(user.bech32(), proxy=context.network_provider.proxy, week=current_week)
print(energy_for_user_in_week)

2023-12-07 14:50:55,550 - ERROR - Exception encountered on view name getUserEnergyForWeek: list index out of range
Traceback (most recent call last):
  File "/home/ovi/projects/dex/mx-sdk-py-exchange/utils/contract_data_fetchers.py", line 63, in _get_hex_view
    return base64_to_hex(result.return_data[0])
                         ~~~~~~~~~~~~~~~~~~^^^
IndexError: list index out of range
2023-12-07 14:50:55,551 - DEBUG - Response content: {'returnData': [], 'returnCode': 'ok', 'returnMessage': '', 'gasUsed': 18446744072212001909}


{}


In [26]:
user_progress = farm_contract.get_current_claim_progress_for_user(user.bech32(), context.network_provider.proxy)
print(user_progress)

2023-12-07 14:51:12,818 - ERROR - Exception encountered on view name getCurrentClaimProgress: list index out of range
Traceback (most recent call last):
  File "/home/ovi/projects/dex/mx-sdk-py-exchange/utils/contract_data_fetchers.py", line 63, in _get_hex_view
    return base64_to_hex(result.return_data[0])
                         ~~~~~~~~~~~~~~~~~~^^^
IndexError: list index out of range
2023-12-07 14:51:12,819 - DEBUG - Response content: {'returnData': [], 'returnCode': 'user error', 'returnMessage': 'storage decode error: input too short', 'gasUsed': 18446744073709551615}


{}


In [27]:
last_week = farm_contract.get_last_active_week_for_user(user.bech32(), context.network_provider.proxy)
print(last_week)

0
